In [20]:
import Parity_Data_Generator
import InputToParityESN
# import ParityToOutputESN
import numpy as np
from pyESN import ESN
# import matplotlib.pyplot as plt
# from mpl_toolkits.mplot3d import Axes3D
import time

start_time = time.time()
rng = np.random.RandomState(42)

N = 50000   # number of datapoints
n = 3       # n-parity
# produce Data
bits, parity, target = Parity_Data_Generator.generateParityData(N, n, randomstate=rng)

# Divide in training and test data
traintest_cutoff = int(np.ceil(0.7 * len(bits)))
train_bits, test_bits = bits[:traintest_cutoff], bits[traintest_cutoff:]
train_parity, test_parity = parity[:traintest_cutoff], parity[traintest_cutoff:]
train_targets, test_targets = target[:traintest_cutoff], target[traintest_cutoff:]

print("--- %s seconds ---" % (time.time() - start_time))

--- 119.39709639549255 seconds ---


In [40]:
def runSomeConfigs(num_neurons=500, spectral_rad, leak_r):
#     print("Spectral-radius:",spectral_rad, "leak-rate:", leak_r)
    nParityESN = ESN(n_inputs=1,
                          n_outputs=1,
                          n_reservoir=num_neurons,  # from 200 onwards test error decreases significantly
                          spectral_radius=spectral_rad,  # from 0.9 the test error is 0.0
                          leak_rate= leak_r,   # adjust leak_rate
                          sparsity=0.95,
                          noise=0.001,
                          input_shift=0,
                          input_scaling=3,  # 3 seems to be fine. Maybe use lower
                          teacher_scaling=1.12,
                          teacher_shift=-0.7,
                          out_activation=np.tanh,
                          inverse_out_activation=np.arctanh,
                          random_state=rng,
                          silent=False)
    # print(np.shape(bits),np.shape(parity),np.shape(target))
    nParityESN.fit(train_bits, train_parity)
    pred_test = nParityESN.predict(test_bits)

    #### Rounding
    better_pred_test = []           # Maybe don't round and return "true" results from pred_test
    for x in pred_test:
        better_pred_test.append(0 if x < 0.5 else 1)
    predictedParity = np.array(better_pred_test).reshape(-1,1)

    print("Testing error")
    test_error = np.sqrt(np.mean((predictedParity - test_parity) ** 2))
    print(test_error)
    return test_error

In [21]:
# num_neurons = [500, 600, 700, 800, 900, 1000]
num_neurons = 500
sp_rad = np.linspace(0.1, 1.0, num=7)
l_rate = np.linspace(0.1, 1.0, num=7)

run_time = time.time()
errors = np.zeros((len(l_rate),len(sp_rad)))

for n in num_neurons:
    for i,l in enumerate(l_rate):
        for j,s in enumerate(sp_rad):
            print("Neurons:", n, "Spectral-radius:", s, "Leak-rate:", l)
            errors[i][j] = runSomeConfigs(n,s,l)
            print("--- %s minutes ---" % (((time.time() - run_time)/60)))
        
plt.imshow(errors, interpolation="nearest", origin="upper")
plt.xlabel("leak-rate")
plt.ylabel("spectral-radius")
plt.colorbar()
plt.show()
        
print("Total time for Testbench:")
print("--- %s seconds ---" % (time.time() - run_time))
print("--- %s minutes ---" % (((time.time() - run_time)/60)))
print("--- %s hours ---" % (((time.time() - run_time)/3600)))

SyntaxError: invalid syntax (<ipython-input-21-b7bd20464c10>, line 17)

# plt.subplot(111)
# plt.plot(errors)
# plt.show()
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
import random

sp_rad = np.linspace(0.1, 1.0, num=7)
l_rate = np.linspace(0.1, 1.0, num=7)
errors = np.zeros((len(l_rate),len(sp_rad)))
for i,l in enumerate(l_rate):
    for j,s in enumerate(sp_rad):
        errors[i][j] = random.randint(-5,6)
print(sp_rad)
print(l_rate)
print(errors)
l_rate, sp_rad = np.meshgrid(l_rate, sp_rad)

fig = plt.figure()
ax = fig.gca(projection='3d')

surf = ax.plot_surface(l_rate, sp_rad, errors, cmap=cm.coolwarm,
                       linewidth=0, antialiased=False)

# Customize the z axis.
# ax.set_zlim(-0.01, 2.01)
ax.zaxis.set_major_locator(LinearLocator(10))
ax.zaxis.set_major_formatter(FormatStrFormatter('%.02f'))

# Add a color bar which maps values to colors.
fig.colorbar(surf, shrink=0.5, aspect=5)

plt.show()

print("### Input-->Parity ###")
run_time = time.time()
# get good configs for a slow ESN
nParityESN = ESN(n_inputs=1,
                          n_outputs=1,
                          n_reservoir=500,  # from 200 onwards test error decreases significantly
                          spectral_radius=0.95,  # from 0.9 the test error is 0.0
                          leak_rate= 0.5,   # adjust leak_rate
                          sparsity=0.95,
                          noise=0.001,
                          input_shift=0,
                          input_scaling=3,  # 3 seems to be fine. Maybe use lower
                          teacher_scaling=1.12,
                          teacher_shift=-0.7,
                          out_activation=np.tanh,
                          inverse_out_activation=np.arctanh,
                          random_state=rng,
                          silent=False)
# print(np.shape(bits),np.shape(parity),np.shape(target))
nParityESN.fit(train_bits, train_parity)
pred_test = nParityESN.predict(test_bits)

#### Rounding
better_pred_test = []           # Maybe don't round and return "true" results from pred_test
for x in pred_test:
    better_pred_test.append(0 if x < 0.5 else 1)
predictedParity = np.array(better_pred_test).reshape(-1,1)

print("Testing error")
print(np.sqrt(np.mean((predictedParity - test_parity) ** 2)))

print("--- %s seconds ---" % (time.time() - run_time))
print("--- %s minutes ---" % (((time.time() - run_time)/60)))